In [1]:
from sklearn import tree
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from skompiler import skompile
from sklearn.tree import _tree

X, y = make_blobs(n_samples=100, centers=3, n_features=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

model = LogisticRegression()
model.fit(X_train, y_train)

df = pd.DataFrame(data=X_train, columns=['x1', 'x2'])
# lin models to sql
expr = skompile(model.predict)
sql = expr.to('sqlalchemy/postgresql')
print(sql)


# tree to sql
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print("SELECT")

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print(node)
            print("CASE WHEN {} <= {}".format(name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("CASE WHEN {} > {}".format(name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print("{}THEN {}".format(indent, tree_.value[node]))

    recurse(0, 1)

    print('FROM')


dt = tree.DecisionTreeClassifier()
dt.fit(X_train, y_train)
tree_to_code(dt, ['x1', 'x2'])

expr = skompile(model.predict)
sql = expr.to('sqlalchemy/postgresql')
print(sql)


WITH _tmp1 AS 
(SELECT data.id AS __id__, -0.6934174201635269 * x1 + -1.3915597270955644 * x2 + -4.289947545295714 AS f1, 1.2604192054711318 * x1 + 0.5078631324223997 * x2 + 2.1603077074084864 AS f2, -0.5670017853076041 * x1 + 0.8836965946731645 * x2 + 2.1296398378872374 AS f3 
FROM data), 
_tmp2 AS 
(SELECT _tmp1.__id__ AS __id__, greatest(greatest(_tmp1.f1, _tmp1.f2), _tmp1.f3) AS _max 
FROM _tmp1)
 SELECT CASE WHEN (_tmp1.f1 = _tmp2._max) THEN 0 WHEN (_tmp1.f2 = _tmp2._max) THEN 1 ELSE 2 END AS y 
FROM _tmp1 JOIN _tmp2 ON _tmp1.__id__ = _tmp2.__id__
SELECT
0
CASE WHEN x2 <= -0.5746956616640091
1
CASE WHEN x2 <= -4.979405879974365
2
CASE WHEN x1 <= 2.2588579654693604
        THEN [[22.  0.  0.]]
CASE WHEN x1 > 2.2588579654693604
        THEN [[0. 2. 0.]]
CASE WHEN x2 > -4.979405879974365
      THEN [[ 0. 19.  0.]]
CASE WHEN x2 > -0.5746956616640091
    THEN [[ 0.  0. 24.]]
FROM
WITH _tmp1 AS 
(SELECT data.id AS __id__, -0.6934174201635269 * x1 + -1.3915597270955644 * x2 + -4.28994754